# Final Colab Notebook: Multi-Agent Vulnerability Detection & Response

**Overview:** This notebook demonstrates a lightweight multi-agent CAI pipeline using Bandit + Semgrep (Scanner), a Hugging Face model for Analyst & Responder, and a Streamlit UI exposed via ngrok.

**Important:** Use synthetic code only. Do NOT paste production secrets. You'll be prompted securely for your Hugging Face token and ngrok authtoken.


In [ ]:
# Install required packages
!pip install --quiet streamlit pyngrok huggingface_hub bandit semgrep reportlab
print('Installed dependencies')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.8/133.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.1/754.1 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.7/100.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.

In [ ]:
from huggingface_hub import InferenceClient
import getpass, os

HF_TOKEN = getpass.getpass('Enter your Hugging Face API token (hf_...): ' )
os.environ['HF_TOKEN'] = HF_TOKEN  # stored for main.py to read
client = InferenceClient(token=HF_TOKEN)
MODEL = 'HuggingFaceH4/zephyr-7b-beta'
print('Hugging Face client ready. Model set to', MODEL)

Enter your Hugging Face API token (hf_...): ··········
Hugging Face client ready. Model set to HuggingFaceH4/zephyr-7b-beta


In [ ]:
%%bash
mkdir -p sample_app
cat > sample_app/vulnerable.py <<'PY'
import os
import sqlite3

# Hardcoded secret (Bandit should flag this)
API_KEY = '12345-SECRET-KEY'

def run_command(user_input):
    # Command injection vulnerability
    os.system('echo ' + user_input)

def insecure_db_lookup(user):
    # SQL injection vulnerability
    conn = sqlite3.connect('users.db')
    cur = conn.cursor()
    query = "SELECT * FROM users WHERE username = '%s'" % user
    cur.execute(query)
    return cur.fetchall()

def main():
    u = input('name: ' )
    run_command(u)
    insecure_db_lookup(u)

if __name__ == '__main__':
    main()
PY
echo 'Created sample_app/vulnerable.py' && ls -l sample_app

Created sample_app/vulnerable.py
total 4
-rw-r--r-- 1 root root 566 Oct  4 18:24 vulnerable.py


In [ ]:
# Write main.py (deterministic pipeline + caching + PDF generator)
%%writefile main.py
import os, json, subprocess, hashlib, re, datetime
from huggingface_hub import InferenceClient
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

# ======================================================
# Setup
# ======================================================
HF_TOKEN = os.environ.get("HF_TOKEN")
if not HF_TOKEN:
    raise RuntimeError("HF_TOKEN not set. Please run: os.environ['HF_TOKEN'] = 'your_token_here'")

client = InferenceClient(token=HF_TOKEN)
MODEL = "HuggingFaceH4/zephyr-7b-beta"

CACHE_DIR = "analysis_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

# ======================================================
# Helpers
# ======================================================
def run_cmd(cmd):
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return p.stdout

def canonicalize(obj):
    return json.dumps(obj, sort_keys=True, separators=(",", ":"), ensure_ascii=False)

def hash_findings(bandit_json, semgrep_json):
    return hashlib.sha256((canonicalize(bandit_json) + canonicalize(semgrep_json)).encode("utf-8")).hexdigest()

def load_cache(key):
    path = os.path.join(CACHE_DIR, f"{key}.json")
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    return None

def save_cache(key, data):
    path = os.path.join(CACHE_DIR, f"{key}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

# ======================================================
# Scanners
# ======================================================
def run_bandit():
    out = run_cmd("bandit -r sample_app -f json -o - 2>/dev/null")
    try:
        return json.loads(out)
    except Exception:
        return {"results": []}

def run_semgrep():
    out = run_cmd("semgrep --config=p/ci sample_app --json 2>/dev/null")
    try:
        return json.loads(out)
    except Exception:
        out2 = run_cmd("semgrep --config auto sample_app --json 2>/dev/null")
        try:
            return json.loads(out2)
        except Exception:
            return {"results": []}

# ======================================================
# Prompts
# ======================================================
SYSTEM_PROMPT = "You are a cybersecurity assistant. Be concise, structured, and factual."

ANALYST_PROMPT_TEMPLATE = """You are an AI Security Analyst.
Analyze the vulnerability reports below from Bandit and Semgrep.

Prioritize vulnerabilities (High/Medium/Low) and explain their impact briefly.
Provide structured, concise output — no exploit details.

Bandit findings:
{bandit}
Semgrep findings:
{semgrep}

Return JSON with:
findings: list of objects with
  id, file, line, tool, issue_text, severity, rationale, remediation.
"""

# === 🧠 AI Recommendation Agent prompt ===
RECOMMENDATION_PROMPT_TEMPLATE = """
You are an AI Security Recommendation Agent.

Based on the analysis below, provide a detailed, developer-focused plan that includes:
1. Concrete fixes or mitigations for each vulnerability.
2. Secure code examples or library/config recommendations.
3. Prioritized order of fixes if multiple issues exist.
4. A short developer message for use in a PR/ticket.
5. Optional: Structured ticket metadata (title, severity, labels).

Analysis JSON:
{analysis}

Return JSON with:
plan: list of objects with id, recommendation, example(optional), priority
ticket: object with title, message, severity, labels
"""

# ======================================================
# LLM Calls
# ======================================================
def call_analyst_and_recommender(bandit_json, semgrep_json):
    key = hash_findings(bandit_json, semgrep_json)
    cached = load_cache(key)
    if cached:
        return cached

    bandit_str = json.dumps(bandit_json.get("results", []), sort_keys=True, ensure_ascii=False)
    semgrep_str = json.dumps(semgrep_json.get("results", []), sort_keys=True, ensure_ascii=False)
    analyst_prompt = ANALYST_PROMPT_TEMPLATE.format(bandit=bandit_str, semgrep=semgrep_str)

    # === Analyst Agent ===
    try:
        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "system", "content": SYSTEM_PROMPT},
                      {"role": "user", "content": analyst_prompt}],
            max_tokens=900, temperature=0.0, top_p=1.0
        )
        text = resp.choices[0].message["content"]
    except Exception:
        text = ""

    try:
        analysis_json = json.loads(re.search(r'(\{.*\})', text, re.DOTALL).group(1))
    except Exception:
        analysis_json = {"findings": []}
        for i, r in enumerate(bandit_json.get("results", []), start=1):
            analysis_json["findings"].append({
                "id": f"bandit-{i}",
                "file": r.get("filename"),
                "line": (r.get("line_range") or None),
                "tool": "bandit",
                "issue_text": r.get("test_name"),
                "severity": "Medium",
                "rationale": r.get("issue_text", "")[:200],
                "remediation": "Review code and follow secure coding guidance."
            })

    # === Recommendation Agent ===
    analysis_text = json.dumps(analysis_json, ensure_ascii=False)
    rec_prompt = RECOMMENDATION_PROMPT_TEMPLATE.format(analysis=analysis_text)

    try:
        resp2 = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "system", "content": SYSTEM_PROMPT},
                      {"role": "user", "content": rec_prompt}],
            max_tokens=1000, temperature=0.0, top_p=1.0
        )
        text2 = resp2.choices[0].message["content"]
    except Exception:
        text2 = ""

    try:
        recommendation_json = json.loads(re.search(r'(\{.*\})', text2, re.DOTALL).group(1))
    except Exception:
        recommendation_json = {"plan": [], "ticket": {"title": "Remediation Plan", "message": "Fix security findings", "severity": "Medium", "labels": ["security"]}}
        for f in analysis_json.get("findings", []):
            recommendation_json["plan"].append({
                "id": f.get("id"),
                "recommendation": f"Fix {f.get('issue_text')} in {f.get('file')}",
                "priority": f.get("severity", "Medium")
            })

    out = {"analysis_json": analysis_json, "recommendation_json": recommendation_json}
    save_cache(key, out)
    return out

# ======================================================
# PDF Report
# ======================================================
def generate_pdf_report(bandit_text, semgrep_text, analysis_text, recommendation_text, output_path="CyberAI_Report.pdf"):
    styles = getSampleStyleSheet()
    doc = SimpleDocTemplate(output_path, pagesize=A4)
    story = []

    story.append(Paragraph("<b>Cybersecurity AI Report</b>", styles["Title"]))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Generated: {datetime.datetime.utcnow().isoformat()} UTC", styles["Normal"]))
    story.append(Spacer(1, 12))

    story.append(Paragraph("<b>Bandit Findings</b>", styles["Heading3"]))
    story.append(Paragraph(f"<pre>{bandit_text[:4000]}</pre>", styles["Code"]))
    story.append(Spacer(1, 12))

    story.append(Paragraph("<b>Semgrep Findings</b>", styles["Heading3"]))
    story.append(Paragraph(f"<pre>{semgrep_text[:4000]}</pre>", styles["Code"]))
    story.append(Spacer(1, 12))

    story.append(Paragraph("<b>AI Analysis</b>", styles["Heading3"]))
    story.append(Paragraph(analysis_text.replace('\n','<br/>'), styles["Normal"]))
    story.append(Spacer(1, 12))

    story.append(Paragraph("<b>AI Recommendations</b>", styles["Heading3"]))
    story.append(Paragraph(recommendation_text.replace('\n','<br/>'), styles["Normal"]))
    story.append(Spacer(1, 12))

    doc.build(story)
    return output_path

# ======================================================
# Coordinator
# ======================================================
def coordinator():
    bandit_json = run_bandit()
    semgrep_json = run_semgrep()

    bandit_sorted = {"results": sorted(bandit_json.get("results", []), key=lambda r: (r.get("filename",""), r.get("test_name","")))}
    semgrep_sorted = {"results": sorted(semgrep_json.get("results", []), key=lambda r: (r.get("path",""), r.get("check_id","")))}

    out = call_analyst_and_recommender(bandit_sorted, semgrep_sorted)
    analysis_json = out.get("analysis_json", {})
    recommendation_json = out.get("recommendation_json") or out.get("responder_json", {})

    # ---- Text formatting ----
    analysis_lines = []
    for f in analysis_json.get("findings", []):
        analysis_lines.append(
            f"[{f.get('severity')}] {f.get('file')}:{f.get('line')} - {f.get('issue_text')} | Remediation: {f.get('remediation')}"
        )
    analysis_text = "\n".join(analysis_lines) or json.dumps(analysis_json, indent=2)

    rec_lines = []
    if isinstance(recommendation_json, dict):
        if "plan" in recommendation_json:
            for step in recommendation_json["plan"]:
                rec_lines.append(f"- {step.get('id')}: {step.get('recommendation')} (Priority: {step.get('priority')})")
        if "ticket" in recommendation_json:
            rec_lines.append(f"\nTicket: {json.dumps(recommendation_json['ticket'], indent=2)}")
    recommendation_text = "\n".join(rec_lines) or json.dumps(recommendation_json, indent=2)

    return bandit_sorted, semgrep_sorted, analysis_text, recommendation_text, analysis_json


Overwriting main.py


In [ ]:
# Write app.py (Streamlit UI)
%%writefile app.py
import streamlit as st
import json
from main import coordinator, generate_pdf_report

st.set_page_config(page_title="CAI Vulnerability Demo (Deterministic)", layout="wide")
st.title("### Multi-Agent CAI Framework Demo (Scanner + Analyst + Responder)")

st.markdown("Click **Run Multi-Agent Scan** to run Bandit+Semgrep, have the AI analyze, and generate a PDF.")

if st.button("🚀 Run Multi-Agent Scan"):
    with st.spinner("Running scanners and AI agents..."):
        bjson, sjson, analysis_text, responder_text, analysis_json = coordinator()

    st.subheader("🔍 Bandit (raw JSON)")
    st.json(bjson)

    st.subheader("🔍 Semgrep (raw JSON)")
    st.json(sjson)

    st.subheader("🧠 AI Analyst")
    st.code(analysis_text, language="text")

    st.subheader("🛠 AI Responder")
    st.code(responder_text, language="text")

    # Generate PDF and offer download
    report_path = generate_pdf_report(json.dumps(bjson, indent=2, ensure_ascii=False),
                                      json.dumps(sjson, indent=2, ensure_ascii=False),
                                      analysis_text,
                                      responder_text,
                                      output_path="CyberAI_Report.pdf")
    with open(report_path, "rb") as f:
        st.download_button("📄 Download report (PDF)", f, file_name="CyberAI_Report.pdf", mime="application/pdf")


Overwriting app.py


In [ ]:
# Launch Streamlit and ngrok (pyngrok). Enter ngrok token when prompted.
!pip install --quiet pyngrok

from pyngrok import ngrok
import getpass, subprocess, time, os

NGROK_TOKEN = getpass.getpass("Enter ngrok authtoken (optional, press Enter to skip): ")
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

# start streamlit in background
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "10000"])
time.sleep(2)
url = ngrok.connect(10000)
print("Open the Streamlit app at:", url.public_url)


Enter ngrok authtoken (optional, press Enter to skip): ··········
Open the Streamlit app at: https://nonprominently-rendible-britni.ngrok-free.dev


In [ ]:

# Kill old tunnels
ngrok.kill()

In [ ]:
#This clears all cached JSONs, ensuring the new Recommendation Agent regenerates deterministic output fresh.
!rm -rf analysis_cache

### Troubleshooting

- If you see `401 Unauthorized` from Hugging Face: re-generate your HF token and re-run the HF token cell.

- If ngrok fails: ensure you set an ngrok token or try re-running the ngrok cell.

- If `main` import fails: ensure you executed the cell that wrote `main.py` before running app cells.

### Quick run checklist

1. Run the install cell.
2. Run the HF token cell and paste token securely.
3. Run the sample_app creation cell.
4. Run the `main.py` and `app.py` write cells.
5. Run the ngrok launch cell and open the printed public URL.
